Motivation: In this notebook, I give an example of how to use my implementation of fast Kronecker inference for GPs with arbitrary likelihoods. See Flaxman et al (2015) "Fast Kronecker Inference in Gaussian Processes with non-Gaussian Likelihoods" for reference.

In [1]:
from kronecker import KroneckerSolver
import kernels as kern
from likelihoods import PoissonLike
import simulator as sim
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from IPython.display import display
init_notebook_mode(connected=True)
from edward import rbf
import GPy
import numpy as np
import itertools
from kernels import RBF

## Simulate some data

First, let's simulate some data on a 2D grid. Given a set of points $\{x_i\}_{i=1}^N$, we generate data via the following model:

$$f \sim ~\mathcal{GP}(\mu(x), K(x, x))$$

$$ y(x_i) \sim ~ \text{Poisson}(f(x_i))$$


For some intuition, here's what the $x_i$'s look like:

In [113]:
X = sim.sim_X_equispaced(D = 2, N_dim = 8)

iplot([go.Scatter(x = X[:,0], y = X[:,1], mode = 'markers', marker=dict(size = 3,))])

Now we draw function values f from a GP with an RBF Kernel, and draw y based on f:

In [136]:
f = sim.sim_f(X, k=RBF(variance=1.0, length_scale=30.0))
y = sim.poisson_draw(f, .5)
iplot([go.Scatter3d(x = X[:,0], y = X[:,1], z=y, mode = 'markers', marker=dict(size = 2,))])

In [137]:
iplot([go.Scatter3d(x = X[:,0], y = X[:,1], z=np.exp(f), mode = 'markers', marker=dict(size = 2,))])

## Inference

We're interested in the following: given y and x, can we infer the function values f(x)? We've made an assumption with the Poisson likelihood here, but this implementation should work with any differentiable likelihood.

Construct a KroneckerSolver object (given a kernel), and run the inference. This should converge within a few Newton iterations.

In [138]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
tfe.enable_eager_execution()

ks = KroneckerSolver(tf.ones([X.shape[0]], tf.float32)*np.mean(np.log(y)), RBF(variance=1.0, length_scale=30.0),
                     PoissonLike(), X, tfe.Variable(y, dtype = tf.float32), tau = 0.5, verbose = True)

ks.run(50)

Iteration:  <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0>
 psi:  tf.Tensor(-42902.4, shape=(), dtype=float32)
step 0.625

Iteration:  tf.Tensor(1, shape=(), dtype=int32)
 psi:  tf.Tensor(-45011.0, shape=(), dtype=float32)
step 1.25

Iteration:  tf.Tensor(2, shape=(), dtype=int32)
 psi:  tf.Tensor(-45178.7, shape=(), dtype=float32)
step 1.25

Iteration:  tf.Tensor(3, shape=(), dtype=int32)
 psi:  tf.Tensor(-45182.3, shape=(), dtype=float32)
step 1.25

Iteration:  tf.Tensor(4, shape=(), dtype=int32)
 psi:  tf.Tensor(-45182.5, shape=(), dtype=float32)
step 1.25

Iteration:  tf.Tensor(5, shape=(), dtype=int32)
 psi:  tf.Tensor(-45182.5, shape=(), dtype=float32)
step 1.25

Iteration:  tf.Tensor(6, shape=(), dtype=int32)
 psi:  tf.Tensor(-45182.5, shape=(), dtype=float32)
step 0.0

Iteration:  tf.Tensor(7, shape=(), dtype=int32)
 psi:  tf.Tensor(-45182.5, shape=(), dtype=float32)
step 0.0



(50,
 <tf.Tensor: id=48553714, shape=(), dtype=int32, numpy=8>,
 <tf.Tensor: id=48546949, shape=(), dtype=float32, numpy=-45182.484>,
 <tf.Tensor: id=48553703, shape=(), dtype=float32, numpy=0.0>)

Plot the inferred function values

In [139]:
iplot([go.Scatter3d(x = X[:,0], y = X[:,1], z= np.exp(np.array(ks.f)), mode = 'markers', marker=dict(size = 2,))])

In [149]:
var = tf.diag_part(cov).numpy()
pred = ks.f.numpy()
lower = pred - 2*np.sqrt(var)
upper = pred + 2*np.sqrt(var)
X0s = np.hstack([X[:,0], X[:,0], X[:,0]])
X1s = np.hstack([X[:,1], X[:,1], X[:,1]])
colors = np.hstack([np.zeros(X.shape[0]), np.ones(X.shape[0]*2)])

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: RuntimeWarning:

invalid value encountered in sqrt

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: RuntimeWarning:

invalid value encountered in sqrt



In [153]:
var

array([  9.59902644e-01,   1.00000000e+00,   1.00000000e+00,
         9.43069756e-01,   1.28346133e+00,   9.99999940e-01,
         9.32406187e-01,   9.72783625e-01,   9.69090044e-01,
         1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
         1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
         1.00000000e+00,   1.00000000e+00,   8.06954145e-01,
         8.81656706e-01,   1.00000000e+00,   9.99999762e-01,
         1.00000012e+00,   2.58864403e+00,   1.00000000e+00,
         1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
         9.68536139e-01,   1.00000000e+00,  -1.86283691e+03,
         1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
         1.00000000e+00,   1.03645480e+00,   1.00000000e+00,
         9.99996722e-01,   1.04737794e+00,   9.64154541e-01,
         1.00000000e+00,   1.17150843e+00,   1.00000000e+00,
         1.00000000e+00,   1.00000000e+00,   1.16107106e+00,
         1.00000000e+00,   1.00000000e+00,   8.78553212e-01,
         1.00000000e+00,

In [152]:
iplot([go.Scatter3d(x = X0s, y = X1s, z= np.hstack([pred, lower, upper]), mode = 'markers', marker=dict(size = 2, color = colors, colorscale = 'Blues'))])

## Partial Grid Structure

In [143]:
indices = np.sort(np.random.choice(X.shape[0], int(X.shape[0]*0.3), replace = False))
X_partial = X[indices]
y_partial = y[indices]
X_partial = X_partial[np.lexsort((X_partial[:,1], X_partial[:,0]))]
X_part_tf = tf.constant(X_partial)
iplot([go.Scatter(x = X_partial[:,0], y = X_partial[:,1], mode = 'markers', marker=dict(size = 5,))])

In [144]:
from grid_utils import fill_grid
import sys

X_full, y_full, obs_idx, imag_idx = fill_grid(X_partial, y_partial)
k_diag = np.ones(len(y_full))
k_diag[imag_idx] = 1e9
y_full_tf = tfe.Variable(y_full, dtype = tf.float32)
mask = tf.less(k_diag, 10)

In [146]:
ks = KroneckerSolver(tf.ones([X_full.shape[0]], tf.float32)*np.mean(np.log(y_full[obs_idx])), RBF(variance=1.0, length_scale=20.0),
                     PoissonLike(), X_full, y_full_tf, 0.5, tf.constant(k_diag, dtype = tf.float32), mask, verbose = True)

ks.run(50)
iplot([go.Scatter3d(x = X_full[obs_idx, 0], y = X_full[obs_idx, 1], z= np.exp(np.array(ks.f)[obs_idx]), mode = 'markers', marker=dict(size = 2))])

Iteration:  <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0>
 psi:  tf.Tensor(-13113.4, shape=(), dtype=float32)
step 3.81469726562e-05

Iteration:  tf.Tensor(1, shape=(), dtype=int32)
 psi:  tf.Tensor(-13606.0, shape=(), dtype=float32)
step 0.0

Iteration:  tf.Tensor(2, shape=(), dtype=int32)
 psi:  tf.Tensor(-13606.0, shape=(), dtype=float32)
step 0.0



In [147]:
test = np.array(list(set(range(X.shape[0])) - set(indices)))
idx = np.zeros(X.shape[0])
idx[indices] = 1
X_test = X[test]
pred = ks.kron_mvp(ks.Ks, ks.alpha) + ks.mu

iplot([go.Scatter3d(x = X[:, 0], y = X[:, 1], z= np.exp(pred), mode = 'markers', marker=dict(size = 2, color = idx))])

In [148]:
W_root = tf.sqrt(ks.W)
W_root_full = tf.diag(W_root)
WKs = tf.stack([tf.squeeze(ks.kron_mvp(ks.Ks, W_root_full[:,i])) for i in range(ks.W.shape[0])], 1)
cov = ks.kron_list(ks.Ks) - [ks.cg(WKs[:,i]) for i in range(ks.W.shape[0])]

In [ ]:
y_new = ks.y.numpy()
y_new[test_idx] = y[test_idx]
k_diag_new = ks.k_diag.numpy()
k_diag_new[test] = 1.0
ks.k_diag = tfe.Variable(k_diag_new)
ks.y = tfe.Variable(y_new)
ks.run(50)
iplot([go.Scatter3d(x = X[:, 0], y = X[:, 1], z= np.exp(np.array(ks.f)), mode = 'markers', marker=dict(size = 2))])